<a href="https://colab.research.google.com/github/prkhr7/prakhar-rawat/blob/master/OSAMAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wtfml
!pip install pretrainedmodels

     |████████████████████████████████| 58 kB 2.7 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=5e6a02ae4f227d8a1a121e7665bf7ac3ca09a60ad73ec22f947bd2c87e4e6c12
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
